In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'unique_product'
AUTHOR = 'g'


In [ ]:
task = Task(
    id=TASK_ID,
    name= "Unique product",
    description="Find the name of any product whose value is different from all others",
    plan=[
         {
        "id": "name",
        "description": "Find the name",
    }, {
        "id": "check",
        "description": "value is different from all others",
    } 
    ],
    sample_input={
       "list":[{"id":0,"value":"Apple"},{"id":1,"value":"Banana"},{"id":2,"value":"Banana"},
               {"id":3,"value":"Banana"} ]
    },
    sample_output="Apple",
    category='First-order logic'
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [5]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def unique_product(list):
  values = [l["value"] for l in list]
  for elem in values:
    if values.count(elem) == 1:
      return elem
  return None
''').load_plan()
imp.execute(task)
imp.save()

In [6]:
func = replace(prototype,
    language='python-functional',
    source='''
def unique_product(list):
  values = [l["value"] for l in list]
  return next(x for x in values if values.count(x) == 1)
''').load_plan()
func.execute(task)
func.save()

In [7]:
pandas = replace(prototype,
    language='python-pandas',
    source='''
def unique_product(list):
  return list[list.groupby('value').value.
    transform(len) == 1].value
''').load_plan()
pandas.execute(task)
pandas.save()

In [4]:
sql = replace(prototype,
    language='sql',
    source='''
SELECT value
FROM list
GROUP BY value
HAVING Count(*) == 1
''').load_plan()
sql.execute(task)
sql.save()

In [9]:
datalog = replace(prototype,
    language='datalog',
    source='''
.decl is_unique(Value:symbol)
is_unique(Value) :-
    list(_,Value), !not_unique(Value).
    
.decl not_unique(Value:symbol)
not_unique(Value) :- 
    list(id1,Value), list(id2,Value), id1 != id2.

unique_product(value) :- is_unique(value).
''').load_plan()
datalog.execute(task)
datalog.save()

In [7]:
program = sql
program.widget(task)

Output()

CodeWidget(program='{"task": "unique_product", "language": "sql", "plan": {"check": [{"line": 4, "start": 0, "…